In [79]:
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [80]:
#TODO: concat these and write to one file.
with open('../data/flutter/flutter_issues_labeled_1.json') as json_data:
    data1 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_2.json') as json_data:
    data2 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_3.json') as json_data:
    data3 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_4.json') as json_data:
    data4 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_5.json') as json_data:
    data5 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_6.json') as json_data:
    data6 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_7.json') as json_data:
    data7 = json.load(json_data)
with open('../data/flutter/flutter_issues_labeled_8.json') as json_data:
    data8 = json.load(json_data)
data1 = [issue for issue in data1 if 'training_labels' in issue]
data2 = [issue for issue in data2 if 'training_labels' in issue]
data3 = [issue for issue in data3 if 'training_labels' in issue]
data4 = [issue for issue in data4 if 'training_labels' in issue]
data5 = [issue for issue in data5 if 'training_labels' in issue]
data6 = [issue for issue in data6 if 'training_labels' in issue]
data7 = [issue for issue in data7 if 'training_labels' in issue]
data8 = [issue for issue in data8 if 'training_labels' in issue]
data = data1 + data2 + data3 + data4 + data5 + data6 + data7 + data8
with open('../data/flutter/flutter_issues_labeled.json', 'w') as json_data:
    json.dump(data, json_data, indent=4)
with open('../data/flutter/flutter_issues_labeled.json') as json_data:
    labeled_issues = json.load(json_data)

UnsupportedOperation: not readable

In [ ]:
num_positive_labels = 0
total_num_labels = 0
for issue in data:
    for assignee in issue['training_labels']:
        if issue['training_labels'][assignee] == 1:
            num_positive_labels += 1
        total_num_labels += 1
print("num positive events: " + str(num_positive_labels))
print("num total events: " + str(total_num_labels))

Now filter out any issues that never end up getting a commit/PR before they are closed. Idea being that these are not 'real' issues.

In [ ]:
num_positive_labels = 0
total_num_labels = 0
for issue in data:
    for assignee in issue['training_labels']:
        if len(issue['matching_prs']) != 0 or len(issue['matching_commits']) != 0:
            if issue['training_labels'][assignee] == 1:
                num_positive_labels += 1
            total_num_labels += 1
print("num positive events: " + str(num_positive_labels))
print("num total events: " + str(total_num_labels))

In [ ]:
person_counts = {}
person_totals = {}
for issue in data:
    for assignee in issue['training_labels']:
        if len(issue['matching_prs']) != 0 or len(issue['matching_commits']) != 0:
            if issue['training_labels'][assignee] == 1:
                person_counts[assignee] = person_counts.get(assignee, 0) + 1
            person_totals[assignee] = person_totals.get(assignee, 0) + 1
person_percents = {}
for person in person_totals:
    person_percents[person] = person_counts.get(person, 0) / person_totals[person]
    
print("Number of successful tasks: ")
print(person_counts)
print("Number of total tasks: ")
print(person_totals)
print("Percentage of tasks completed: ")
print(person_percents)

In [ ]:
#TODO: compare characteristics of issues that are successfully completed.